<a href="https://colab.research.google.com/github/cmvera16/FoundationsModels/blob/main/stable_diffusion_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instalación de librerías necesarias

In [15]:
!pip install diffusers transformers torch accelerate pillow

## 2. Verificar disponibilidad de GPU

In [16]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: True


## 3. Cargar librerías

In [17]:
import os
from PIL import Image
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, StableDiffusionImg2ImgPipeline
import torch


## 4. Definir rutas

In [18]:
input_dir = r'/content/custom_dataset'
output_dir = r'/content/generated_images'
variation_dir = r'/content/outputs/variations'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(variation_dir, exist_ok=True)

## 5. Cargar modelo base de Hugging Face

In [19]:
model_id = "stabilityai/stable-diffusion-2-1-base"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    token=None  # Use token=None for public models
).to("cuda" if torch.cuda.is_available() else "cpu")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

#####  Cambiar el scheduler (opcional, ya viene por defecto)

In [20]:
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

## 6. Generar imágenes nuevas desde prompts

In [21]:
prompts = [
    "A high-resolution landscape photograph of a mountain valley at sunrise",
    "A detailed forest with mist and morning light, cinematic lighting",
    "A wide view of a desert with dramatic clouds and shadows",
    "A colorful beach at sunset with reflections in the water",
]

for i, prompt in enumerate(prompts):
    image = pipe(prompt=prompt, guidance_scale=7.5, num_inference_steps=50).images[0]
    image.save(os.path.join(output_dir, f"generated_image_{i+1}.png"))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

## 7. Generar variaciones desde imágenes base (Image-to-Image)

In [22]:
from diffusers import StableDiffusionImg2ImgPipeline

img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
).to("cuda" if torch.cuda.is_available() else "cpu")

variation_prompt = "A futuristic version of this landscape with neon lighting and surreal atmosphere"



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

##### Seleccionamos una imagen como ejemplo

In [23]:
image_path = os.path.join(input_dir, os.listdir(input_dir)[0])
init_image = Image.open(image_path).convert("RGB").resize((512, 512))

image_variation = img2img_pipe(
    prompt=variation_prompt,
    image=init_image,
    strength=0.75,
    guidance_scale=7.5
).images[0]

image_variation.save(os.path.join(variation_dir, "variation_1.png"))

IndexError: list index out of range

### Upload your image for Image-to-Image Generation

In [24]:
from google.colab import files
import os

input_dir = '/content/custom_dataset'

# Clean the directory in case you are re-running this cell
for f in os.listdir(input_dir):
    os.remove(os.path.join(input_dir, f))

uploaded = files.upload()

for fn in uploaded.keys():
  os.rename(fn, os.path.join(input_dir, fn))

print(f"Uploaded {len(uploaded.keys())} files to {input_dir}")

Saving img_flores(1).jpg to img_flores(1).jpg
Saving img_flores(2).jpg to img_flores(2).jpg
Saving img_flores(3).jpg to img_flores(3).jpg
Saving img_flores(4).jpg to img_flores(4).jpg
Saving img_flores(5).jpg to img_flores(5).jpg
Saving img_flores(6).jpg to img_flores(6).jpg
Saving img_flores(7).jpg to img_flores(7).jpg
Saving img_flores(8).jpg to img_flores(8).jpg
Saving img_flores(9).jpg to img_flores(9).jpg
Saving img_flores(10).jpg to img_flores(10).jpg
Uploaded 10 files to /content/custom_dataset
